# noobai-XL-1.1 Text-to-Image Model Setup

This Colab notebook sets up the `noobai-XL-1.1` model for text-to-image generation with customizable settings. You can adjust sampler methods, sampling steps, seeds, CFG scale, image size, and apply the `hires fix 8x-NKMD-Superscale` for high-resolution outputs.

In [1]:
# Install necessary libraries
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip install transformers diffusers accelerate
!pip install git+https://github.com/huggingface/diffusers.git
!pip install --upgrade transformers
!pip install pillow

## Import Libraries

In [2]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
import transformers
from PIL import Image
import os

## Configure the Model and Tokenizer

In [3]:
# Clone the model repository
!git lfs install
!git clone https://huggingface.co/Laxhar/noobai-XL-1.1

# Modify tokenizer configuration to remove character limit
!sed -i 's/"max_length": 77/"max_length": 1024/' noobai-XL-1.1/tokenizer/tokenizer_config.json
!sed -i 's/"max_length": 77/"max_length": 1024/' noobai-XL-1.1/tokenizer_2/tokenizer_config.json

## Load the Model

Set up the scheduler and load the Stable Diffusion pipeline with the modified tokenizer.

In [4]:
# Choose the device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Set up the scheduler
scheduler = EulerAncestralDiscreteScheduler.from_pretrained("Laxhar/noobai-XL-1.1", subfolder="scheduler")

# Load the pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "Laxhar/noobai-XL-1.1",
    scheduler=scheduler,
    revision="fp16",
    torch_dtype=torch.float16,
    use_auth_token=True  # Make sure you have access to the model
).to(device)

## Define Utility Functions

In [5]:
def generate_image(prompt, sampler='euler', steps=50, seed=None, cfg_scale=7.5, image_size=(512, 512)):
    
    # Set the sampler
    if sampler == 'euler':
        scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
    elif sampler == 'ddim':
        scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
    # Add more samplers if needed
    else:
        raise ValueError(f"Unsupported sampler: {sampler}")
    
    pipe.scheduler = scheduler
    
    # Set seed for reproducibility
    if seed is not None:
        generator = torch.Generator(device=device).manual_seed(seed)
    else:
        generator = None
    
    # Generate the image
    with autocast(device):
        image = pipe(
            prompt,
            num_inference_steps=steps,
            guidance_scale=cfg_scale,
            height=image_size[1],
            width=image_size[0],
            generator=generator
        ).images[0]
    return image

## High-Resolution Fix with 8x-NKMD-Superscale

This section applies a high-resolution fix using the `8x-NKMD-Superscale` model. You can adjust the scale, creativity, and steps for the superscaling process.

In [6]:
# Clone the SuperScale repository
!git clone https://github.com/NKMD-Development/superscale-8x.git

# Load the SuperScale model (modify as per actual model usage)
# This is a placeholder for the actual SuperScale implementation
# You may need to install specific dependencies or use a different library
# For demonstration, we'll assume a simple upscale using PIL
def superscale_image(image, scale=8, creativity=1.0, steps=10):
    
    # Placeholder for the actual SuperScale process
    # Here we'll just resize the image as an example
    new_size = (image.width * scale, image.height * scale)
    upscaled_image = image.resize(new_size, Image.LANCZOS)
    return upscaled_image

## User Inputs

Set your desired parameters below.

In [7]:
# User-configurable parameters
prompt = "A serene landscape with mountains and a river"
sampler = "euler"  # Options: 'euler', 'ddim'
steps = 50
seed = 42
cfg_scale = 7.5
image_size = (512, 512)

# High-resolution fix parameters
apply_superscale = True
scale_factor = 8
creativity = 1.0
superscale_steps = 10

## Generate and Display the Image

In [8]:
# Generate the image
image = generate_image(
    prompt=prompt,
    sampler=sampler,
    steps=steps,
    seed=seed,
    cfg_scale=cfg_scale,
    image_size=image_size
)

# Apply high-resolution fix if enabled
if apply_superscale:
    image = superscale_image(
        image,
        scale=scale_factor,
        creativity=creativity,
        steps=superscale_steps
    )

# Display the image
image

<PIL.Image.Image image mode=RGB size=4096x4096 at 0x7F8C3C2E9C10>

## Save the Image

In [9]:
# Save the image to the local filesystem
output_path = "generated_image.png"
image.save(output_path)
print(f"Image saved to {output_path}")

## Additional Resources

- [Hugging Face Diffusers](https://huggingface.co/docs/diffusers/index)
- [Stable Diffusion Documentation](https://huggingface.co/docs/diffusers/stable_diffusion)
- [Transformers Documentation](https://huggingface.co/docs/transformers/index)